# Setup

In [1]:
import requests
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
import nest_asyncio
from dotenv import load_dotenv
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
)
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_ollama.llms import OllamaLLM
from langchain_openai import ChatOpenAI


nest_asyncio.apply()

# Load environment variables from .env file
load_dotenv()

True

# DIP Data

In [2]:
base_url = "https://search.dip.bundestag.de/api/v1"
api_key = "I9FKdCn.hbfefNWCY336dL6x62vfwNKpoN2RZ1gp21"

In [3]:
# Get all documents metadata
metadata_endpoint = "plenarprotokoll"
headers = { "Authorization": f"ApiKey {api_key}"}

metadata_url = f"{base_url}/{metadata_endpoint}"
metadatas = requests.get(metadata_url, headers=headers).json()["documents"]

In [4]:
# Get Text of one document
fulltext_endpoint = "plenarprotokoll-text"
params = {"format": "xml"}
def get_text(document_id: str) -> str:
    fulltext_url = f"{base_url}/{fulltext_endpoint}/{document_id}"
    response = requests.get(fulltext_url, headers=headers, params=params)
    return response.content


In [55]:
def get_rede_from_rede_tag(rede_tag):
    # Get redner
    redner_tag = rede_tag.find('redner')
    first_name = redner_tag.find('vorname').text
    last_name = redner_tag.find('nachname').text
    party = redner_tag.find('fraktion').text if redner_tag.find('fraktion') else None
    rolle = redner_tag.find('rolle_lang').text if redner_tag.find('rolle_lang') else None
    redner = {
        "last_name": last_name,
        "first_name": first_name,
        "party": party,
        "rolle": rolle
    }
    redner_tag.decompose()

    # Delete comments
    for kommentar in rede_tag.find_all('kommentar'):
        kommentar.decompose()

    return {"redner": redner, "rede": rede_tag.text} 


In [56]:
from bs4 import BeautifulSoup
import re

xml_url = metadatas[3]["fundstelle"]["xml_url"]
xml_content = requests.get(xml_url).content

soup = BeautifulSoup(xml_content, features='xml')

In [57]:
all_reden_tags = soup.find_all("rede")
reden = []

for rede_tag in all_reden_tags:
    rede = get_rede_from_rede_tag(rede_tag)
    reden.append(rede)
reden

[{'redner': {'last_name': 'Wissing',
   'first_name': 'Volker',
   'party': None,
   'rolle': 'Bundesminister für Digitales und Verkehr'},
  'rede': '\nDr.\xa0Volker Wissing, Bundesminister für Digitales und Verkehr:\nHerzlichen Dank.\xa0– Frau Präsidentin! Liebe Kolleginnen und Kollegen! Wir haben uns für den Gigabitausbau in unserem Land ein ehrgeiziges Ziel gesetzt: Wir wollen eine flächendeckende Versorgung mit Glasfaser bis zum Jahr\xa02030 und den neuesten Mobilfunkstandard überall dort, wo Menschen leben, arbeiten oder unterwegs sind. Um dieses Ziel zu erreichen, haben wir in unserer Gigabitstrategie verschiedene Maßnahmen, die wirken, beschlossen.\nEine Maßnahme, die noch kommt, ist der Gesetzentwurf, der gerade vorliegt. Mit diesem Gesetz beschleunigen wir den Netzausbau, der erfreulicherweise in den vergangenen zwei Jahren ordentlich an Fahrt aufgenommen hat.\n\nWie stark er an Fahrt aufgenommen hat, bescheinigte uns neulich die EU-Kommission, indem sie von einer spektakuläre

# RAG

In [5]:
# Create ParentDocumentRetriever
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=HuggingFaceEmbeddings(model_name=embedding_model)
)
store = InMemoryStore()

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

/home/tim/projects/dip/dip-rag/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [6]:
def get_langchain_document_from_protocol_id(protocol_id: str) -> Document:
    metadata_url = f"{base_url}/{metadata_endpoint}/{protocol_id}"
    metadata = requests.get(metadata_url, headers=headers).json()
    

    retrieval_metadata = {
        "dokument_id": protocol_id,
        "dokument_art": metadata["dokumentart"],
        "dokument_nummer": metadata["dokumentnummer"],
        "titel": metadata["titel"],
        "datum": metadata["datum"],
    }
    retrieval_text = get_text(document_id=protocol_id)

    return Document(page_content=retrieval_text, metadata=retrieval_metadata)

In [7]:
# Ingest one protocol
index = 10
protocol_id = metadatas[index]["id"]

document = get_langchain_document_from_protocol_id(protocol_id=protocol_id)
retriever.add_documents(documents=[document])

In [8]:
# Load Generation Model
llama_model_3 = "meta-llama/Llama-3.2-3B-Instruct"
mixtral_model = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llama_2b_70b = "meta-llama/Llama-2-70b-chat-hf"
openai_gpt_3_5 = "gpt-3.5-turbo"

llm = HuggingFaceEndpoint(
    repo_id=mixtral_model,
    task="text-generation",
    max_new_tokens=1024,
)

chat_model = ChatOpenAI(model=openai_gpt_3_5)
#chat_model = ChatHuggingFace(llm=llm)
fact_check_model = ChatHuggingFace(llm=llm)
#chat_model = OllamaLLM(model="mistral")
#fact_check_model = OllamaLLM(model="llama3.2:1b")


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/tim/.cache/huggingface/token
Login successful


In [9]:
def format_docs(documents: list[Document]) -> str:
    return "\n\n".join(document.page_content for document in documents)

In [16]:
def get_rag_answer(prompt: str):
    documents = retriever.invoke(prompt, k=4)
    context = format_docs(documents=documents)

    simple_rag_prompt = (
        "Beantworte die Frage mit Hilfe der folgenden Kontextinformationen. "
        "Wenn du die Antwort nicht weißt, sag einfach, dass du die Antwort nicht kennst. "
        "Verwende zu Beantwortung nur die Informationen im Kontext. Verwende kein externes Wissen. "
        "Verwende maximal fünf Sätze und fasse die Antwort kurz zusammen."
        f"Frage: {prompt}"
        f"Kontext: {context}"
        "Antwort: " 
        )

    simple_system_prompt =  "Du bist ein Assistent für die Beantwortung von Fragen bezüglich Plenarsitzungen des Deutschen Bundestags."

    messages = [
        SystemMessage(content=simple_system_prompt),
        HumanMessage(
            content=simple_rag_prompt
        ),
    ]

    answer = chat_model.invoke(messages).content
    return (answer, documents)

In [17]:
# Self Fact-Checking
def check_facts(answer:str, documents: list[Document])-> tuple[bool, str]:
    context = format_docs(documents=documents)

    fact_checking_prompt = (
        "Du hast die Aufgabe, herauszufinden, ob die Hypothese begründet ist und mit den Beweisen übereinstimmt. "
        "Verwende nur den Inhalt der Beweise und stütze dich nicht auf externes Wissen. "
        f"Antworte mit ja/nein. Beweise: {context} "
        f"Hypothese: {answer}: "
        "Antwort: "
    )

    ai_msg = fact_check_model.invoke([
        HumanMessage(
            content=fact_checking_prompt
        ),
    ]).content
    is_okay = ai_msg.lower().strip().startswith("ja")
    return (is_okay, ai_msg)


In [19]:
# Ask RAG
prompt = "Wer nannte Herrn Steffen einen Hetzer?"
(answer, documents) = get_rag_answer(prompt=prompt)
(is_based_on_facts, fact_checking_answer) = check_facts(answer=answer, documents=documents)

print(f"Fact-Check: {'OKAY' if is_based_on_facts else 'NOT OKAY'}")
print(answer)

{}
Fact-Check: OKAY
Stephan Brandner (AfD) nannte Herrn Steffen einen "Hetzer".


In [13]:
fact_checking_answer

' Ja, die Hypothese wird bestätigt. Aus dem Stenografischen Bericht geht hervor, dass Stephan Brandner (AfD) Herrn Steffen als "Hetzer" bezeichnet hat.'

In [ ]:
# Build Evaluation Dataset

evaluation_questions = [
    "Wer nannte Herrn Steffen einen Hetzer?"
]

evaluation_answers = [
    "Stephan Brandner (AfD) nannte Herrn Steffen einen 'Hetzer'."
]